In [18]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
website='https://www.adamchoi.co.uk/teamgoals/detailed'
path='/home/bahar/Descargas/chromedriver'


In [ ]:
import time
websiteTF='https://www.cinemarkca.com/index'

driver = webdriver.Chrome(path)
driver.get(websiteTF)

time.sleep(2)

cerrarVentanacine= driver.find_element('id','modal-theatre-select')
cerrarcine= cerrarVentanacine.find_element(By.CLASS_NAME,'change-theatre-button')
cerrar=cerrarcine.find_element('xpath','//button[@class="btn-primary next"]')
cerrar.click()

country= driver.find_element('id','accordion')
guatemala=country.find_element('xpath','//div[@class="panelTitle accordion-toggle"]')
guatemala.click()
time.sleep(2)
cinesGuate=guatemala.find_element('xpath','//ul[@class="panelBody"]')
arkadia= cinesGuate.find_element(By.CSS_SELECTOR, 'a')
arkadia.click()
time.sleep(2)

cerrarVentanacine1= driver.find_element('id','modal-theatre-select')
cerrarcine1= cerrarVentanacine1.find_element(By.CLASS_NAME,'change-theatre-button')
cerrar1=cerrarcine1.find_element('xpath','//button[@class="btn-primary next"]')
cerrar1.click()
time.sleep(5)
#carteleraArkadia=driver.find_element('xpath','//ul[@class="box-menu grid-center-content"]')
#cartelera=carteleraArkadia.find_element('xpath','//a[@href="https://www.cinemarkca.com/guatemala/cine?tag=2202"]')
